In [1]:
from qiskit import QuantumCircuit, qasm3
from quantum_chip import QuantumChip
from hardware import HardwareParams
from qrmove_compiler import QRMoveCompiler
from utils import get_quantum_circuit
import math
from func_timeout import func_timeout, FunctionTimedOut
import csv


def experiment(quantum_alg, qubit_num, extra_qubit_num, MRP_CP_ratio):
    """
    quantum_alg: 量子算法
    qubit_num: 量子比特数量
    extra_qubit_ratio: 额外比特比例
    MRP_CP_ratio: MRP/CP 比例
    """
    try:
        quantum_circuit: QuantumCircuit = get_quantum_circuit(quantum_alg, qubit_num)
        quantum_chip = QuantumChip("square", 50)
        hardware_param = HardwareParams(
            time_1q=300.0,  # 单比特门时间
            time_2q=300.0,  # 双比特门时间
            time_meas=300 * MRP_CP_ratio,  # 测量时间
            time_reset=0,  # 重置时间
        )
        qmc = QRMoveCompiler(
            quantum_circuit,
            quantum_chip,
            hardware_param,
            math.ceil(extra_qubit_num),
        )
        (
            start_depth,
            start_width,
            qr_map_depth,
            qr_map_width,
            qr_move_depth,
            qr_move_width,
        ) = qmc.compile_program()

        return (
            start_depth,
            start_width,
            qr_map_depth,
            qr_map_width,
            qr_move_depth,
            qr_move_width,
        )

    except Exception as e:
        print(f"Experiment failed with error: {e}")
        return 0, 0, 0, 0, 0, 0


In [2]:
# Default Settings
# quantum_alg: bv
# MRP_CP_ratio: 10
# extra_qubit_ratio: 15%
# qubit_num: 30
# bv qaoa mod mul qram rd sym xor
# for alg in ["xor"]:
for alg in ["bv", "qaoa", "mul", "rd"]:
    for qubit_num in [10, 20, 30, 40, 50]:
        extra_num = 2
        mrp_ratio = 10
        print(f"alg: {alg}, qubit_num: {qubit_num}")
        for _ in range(5):
            (
                start_depth,
                start_width,
                qr_map_depth,
                qr_map_width,
                qr_move_depth,
                qr_move_width,
            ) = experiment(alg, qubit_num, extra_num, mrp_ratio)
            print(
                f"运行结果: start_depth {start_depth}, start_width {start_width}, qr_map_depth {qr_map_depth}, qr_map_width {qr_map_width}, qr_move_depth {qr_move_depth}, qr_move_width {qr_move_width}"
            )

alg: bv, qubit_num: 10
运行结果: start_depth 14, start_width 10, qr_map_depth 41, qr_map_width 2, qr_move_depth 22, qr_move_width 3
运行结果: start_depth 14, start_width 10, qr_map_depth 41, qr_map_width 2, qr_move_depth 23, qr_move_width 3
运行结果: start_depth 14, start_width 10, qr_map_depth 41, qr_map_width 2, qr_move_depth 22, qr_move_width 3
运行结果: start_depth 14, start_width 10, qr_map_depth 41, qr_map_width 2, qr_move_depth 23, qr_move_width 3
运行结果: start_depth 14, start_width 10, qr_map_depth 41, qr_map_width 2, qr_move_depth 22, qr_move_width 3
alg: bv, qubit_num: 20
运行结果: start_depth 19, start_width 20, qr_map_depth 91, qr_map_width 2, qr_move_depth 44, qr_move_width 4
运行结果: start_depth 19, start_width 20, qr_map_depth 91, qr_map_width 2, qr_move_depth 35, qr_move_width 4
运行结果: start_depth 19, start_width 20, qr_map_depth 91, qr_map_width 2, qr_move_depth 34, qr_move_width 4
运行结果: start_depth 19, start_width 20, qr_map_depth 91, qr_map_width 2, qr_move_depth 43, qr_move_width 4
运行结果: sta

In [ ]:
def experiment_var_qubit_num():
    from concurrent.futures import ThreadPoolExecutor, as_completed
    from func_timeout import func_timeout, FunctionTimedOut

    # 存储每个参数组合的任务
    all_tasks = []

    # 创建所有参数组合
    qubit_nums = [10, 20, 30, 40, 50]
    algorithms = ["bv", "vqa", "qaoa", "qft", "mul", "rd", "xor"]

    for qubit_num in qubit_nums:
        for alg in algorithms:
            MRP_CP_ratio = 10
            extra_qubit_ratio = 0.15
            all_tasks.append((alg, qubit_num, MRP_CP_ratio, extra_qubit_ratio))

    # 为每个参数组合执行50次实验
    def run_experiments_for_params(alg, qubit_num, MRP_CP_ratio, extra_qubit_ratio):
        results = []
        for _ in range(50):
            try:
                # 使用 func_timeout 设置超时时间为10分钟(600秒)
                (
                    start_depth,
                    start_width,
                    qr_map_depth,
                    qr_map_width,
                    qr_move_depth,
                    qr_move_width,
                ) = func_timeout(
                    60 * 30,  # 超时时间（秒）
                    experiment,
                    args=(alg, qubit_num, extra_qubit_ratio * qubit_num, MRP_CP_ratio),
                )

                # 如果实验成功（不是全零结果），则添加到结果列表中
                if not (
                    start_depth == 0
                    and start_width == 0
                    and qr_map_depth == 0
                    and qr_map_width == 0
                    and qr_move_depth == 0
                    and qr_move_width == 0
                ):
                    results.append(
                        (
                            start_depth,
                            start_width,
                            qr_map_depth,
                            qr_map_width,
                            qr_move_depth,
                            qr_move_width,
                        )
                    )
            except FunctionTimedOut:
                print(f"Experiment timed out for {alg} with {qubit_num} qubits")
            except Exception as e:
                print(f"Experiment failed with error: {e}")
                # 继续执行下一次实验

        return alg, qubit_num, MRP_CP_ratio, extra_qubit_ratio, results

    # 使用线程池并发执行所有实验
    results_dict = {}
    with ThreadPoolExecutor(max_workers=96) as executor:
        # 提交所有任务
        future_to_params = {
            executor.submit(
                run_experiments_for_params,
                alg,
                qubit_num,
                MRP_CP_ratio,
                extra_qubit_ratio,
            ): (alg, qubit_num, MRP_CP_ratio, extra_qubit_ratio)
            for alg, qubit_num, MRP_CP_ratio, extra_qubit_ratio in all_tasks
        }

        # 收集结果
        for future in as_completed(future_to_params):
            alg, qubit_num, MRP_CP_ratio, extra_qubit_ratio, valid_results = (
                future.result()
            )
            results_dict[(alg, qubit_num)] = valid_results

    # 处理和打印结果，并写入文件
    # 准备写入CSV文件
    with open("experiment_results.csv", "w", newline="") as csvfile:
        fieldnames = [
            "algorithm",
            "qubit_num",
            "extra_qubit_ratio",
            "MRP_CP_ratio",
            "avg_start_depth",
            "avg_start_width",
            "avg_qr_map_depth",
            "avg_qr_map_width",
            "avg_qr_move_depth",
            "avg_qr_move_width",
            "valid_experiment_count",
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for qubit_num in qubit_nums:
            for alg in algorithms:
                MRP_CP_ratio = 10
                extra_qubit_ratio = 0.15
                valid_results = results_dict.get((alg, qubit_num), [])

                print(
                    f"-- 参数: {alg} 算法, 额外比特比例 {extra_qubit_ratio}, 量子比特数量 {qubit_num} --"
                )

                # 计算平均值
                if valid_results:
                    avg_start_depth = sum(r[0] for r in valid_results) / len(
                        valid_results
                    )
                    avg_start_width = sum(r[1] for r in valid_results) / len(
                        valid_results
                    )
                    avg_qr_map_depth = sum(r[2] for r in valid_results) / len(
                        valid_results
                    )
                    avg_qr_map_width = sum(r[3] for r in valid_results) / len(
                        valid_results
                    )
                    avg_qr_move_depth = sum(r[4] for r in valid_results) / len(
                        valid_results
                    )
                    avg_qr_move_width = sum(r[5] for r in valid_results) / len(
                        valid_results
                    )

                    print(
                        f"平均值: {avg_start_depth}, {avg_start_width}, {avg_qr_map_depth}, {avg_qr_map_width}, {avg_qr_move_depth}, {avg_qr_move_width}"
                    )
                    print(f"有效实验次数: {len(valid_results)}")

                    # 写入CSV文件
                    writer.writerow(
                        {
                            "algorithm": alg,
                            "qubit_num": qubit_num,
                            "extra_qubit_ratio": extra_qubit_ratio,
                            "MRP_CP_ratio": MRP_CP_ratio,
                            "avg_start_depth": avg_start_depth,
                            "avg_start_width": avg_start_width,
                            "avg_qr_map_depth": avg_qr_map_depth,
                            "avg_qr_map_width": avg_qr_map_width,
                            "avg_qr_move_depth": avg_qr_move_depth,
                            "avg_qr_move_width": avg_qr_move_width,
                            "valid_experiment_count": len(valid_results),
                        }
                    )
                else:
                    print("所有实验都失败了，无法计算平均值")
                    # 写入CSV文件，平均值为0
                    writer.writerow(
                        {
                            "algorithm": alg,
                            "qubit_num": qubit_num,
                            "extra_qubit_ratio": extra_qubit_ratio,
                            "MRP_CP_ratio": MRP_CP_ratio,
                            "avg_start_depth": 0,
                            "avg_start_width": 0,
                            "avg_qr_map_depth": 0,
                            "avg_qr_map_width": 0,
                            "avg_qr_move_depth": 0,
                            "avg_qr_move_width": 0,
                            "valid_experiment_count": 0,
                        }
                    )


# 调用实验函数
experiment_var_qubit_num()